In [3]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate
from scipy.integrate import quad

from helpers import figures


def thermal_conductivity_NIST(T):
    """
    NISTmonograph177-Propertiesofcopperandcopperalloysatcryogenictemperatures--OCR.pdf
    :param T:
    :return:
    """
    tin_concentration = 5  # in percent
    a = 0.4145
    b = 1.563
    c = -0.2285
    d = -0.3234
    e = 0.02500
    return 10 ** (
        a
        + b * np.log10(T)
        - c * np.log10(T) ** 2
        - d * tin_concentration
        + e * tin_concentration**2
    )


def thermal_conductivity(T):
    """
    Data from https://ntrs.nasa.gov/api/citations/20090032058/downloads/20090032058.pdf
    """
    a, b, c, d, e, f, g, h, i = (
        -10.9482,
        28.4752,
        -32.3378,
        20.9036,
        -8.05399,
        1.90329,
        -0.271774,
        0.0215998,
        -7.35095e-4,
    )
    return math.e ** (
        a
        + b * np.log(T)
        + c * np.log(T) ** 2
        + d * np.log(T) ** 3
        + e * np.log(T) ** 4
        + f * np.log(T) ** 5
        + g * np.log(T) ** 6
        + h * np.log(T) ** 7
        + i * np.log(T) ** 8
    )


def specific_heat(T):
    print("____")
    tin_concentration = 5
    debye = lambda x: math.e**x * x**4 / (math.e**x - 1) ** 2

    gamma = (
        0.698
        + 0.00911 * tin_concentration
        - 0.00128 * tin_concentration**2
        + 6.54e-5 * tin_concentration**3
    )
    theta_D = 344 - 3.11 * tin_concentration
    M = 1.187 * tin_concentration + 0.6354 * (100 - tin_concentration)  # gramm/mol

    x_m = theta_D / T
    debye_result, err = quad(debye, 0, x_m)
    debye_value = 7.48e4 * debye_result / (x_m**3)

    if 1.5 < T <= 4:
        return gamma * T / M + 1.94e6 * (T / theta_D) ** 3 / M
    c_i = debye_value / M
    c_e = gamma * T / M
    return c_e + c_i


def _fit_thermal_expansion():
    data = pd.read_csv("cte.csv", index_col="temp")["cte"]
    return scipy.interpolate.interp1d(x=data.index, y=data, assume_sorted=True, kind="cubic")


def thermal_expansion(T):
    """
    data from https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=913059
    :param T: temperature in kelvin
    :return: coefficient of thermal expansion in 1/Kelvin
    """
    return _fit_thermal_expansion()(T) * 1e-6


print(thermal_expansion(11))


# print([(T, specific_heat(T=T)) for T in np.linspace(4,104, num=21)])
# print(specific_heat(T=77))
# print(specific_heat(T=295))

8.050163738933644e-08


In [8]:
figures.initialize()
fig, ax1 = plt.subplots(figsize=figures.set_size(subplots=(1, 2)))

ax2 = ax1.twinx()
xs = np.logspace(np.log10(4), np.log10(300))
ys = thermal_conductivity(xs)

ax1.semilogx(xs, ys, label="Heat conductivity")
ax2.loglog(xs, [specific_heat(T) for T in xs], label="Specific heat", color="C2")

xlabel = plt.xlabel("Temperature (K)")
ylabel = ax1.set_ylabel("Thermal conductivity\n(W/m·K)")
ylabel2 = ax2.set_ylabel("Specific heat capacity\n(J/kg·K)")
ylabel2.set_color("C2")
ax1.set_xlabel("Temperature (K)")
ax2.tick_params(axis="y", which="both", colors="C2")
ax2.spines["right"].set_color("C2")
# fig.legend(loc='lower right', bbox_to_anchor=(0.85, 0.15), frameon=False)
ax1.grid(which="major", axis="x")
fig.savefig(
    "thermal-properties.ignore.pdf",
    backend="pgf",
)
fig.savefig(
    "thermal-properties.pgf",
    backend="pgf",
)
plt.close(fig)

____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____


2.718281828459045